# 07 — Module 9: Daily report (end-to-end pipeline)

This notebook runs the full daily pipeline and exports a CSV.

Pipeline:
1) build eligible universe
2) rank top N
3) compute entry signals
4) compute position sizing
5) generate a final report table

In [1]:
# If running from repo root and editable install is not done:
# pip install -e ".[dev]"

import pandas as pd
pd.set_option("display.width", 140)
pd.set_option("display.max_columns", 50)

> Tip: many modules assume the benchmark **SPY** is present (for Relative Strength).
> When using real tickers, include SPY:
>
> `tickers = ["AAPL","MSFT","NVDA","SPY"]`

In [2]:
from swing_screener.data.market_data import fetch_ohlcv, MarketDataConfig
from swing_screener.reporting.report import build_daily_report, export_report_csv, today_actions, ReportConfig
from swing_screener.risk.position_sizing import RiskConfig

tickers = [
    "AAPL",
    "MSFT",
    "NVDA",
    "AMZN",
    "META",
    "GOOGL",
    "TSLA",
    "AMD",
    "NFLX",
    "AVGO",
    "INTC",
    "ORCL",
    "CRM",
    "QCOM",
    "ADBE",
    "CSCO",
    "SHOP",
    "UBER",
    "ABNB",
    "SPY",
]
ohlcv = fetch_ohlcv(tickers, MarketDataConfig(start="2022-01-01"))

cfg = ReportConfig(
    risk=RiskConfig(account_size=500, risk_pct=0.01, k_atr=2.0, max_position_pct=0.60),
    only_active_signals=False,
)

report = build_daily_report(ohlcv, cfg)
report.head(20)

,rank,score,last,atr14,atr_pct,mom_6m,mom_12m,rs_6m,trend_ok,dist_sma50_pct,dist_sma200_pct,signal,breakout_level,ma20_level,entry,stop,shares,position_value,realized_risk
INTC,1,1.0,47.290001,2.205714,4.664229,1.063264,1.469452,0.941571,True,22.857338,67.793513,breakout,45.549999,38.968,47.29,42.88,1,47.29,4.41


In [3]:
print(today_actions(report, max_rows=5))

TODAY ACTIONS (tradable signals):
 - INTC: breakout | entry~47.29 | stop~42.88 | shares=1 | risk~4.41


In [ ]:
export_path = export_report_csv(report, "out/daily_report.csv")
export_path